In [15]:
import os
from requests import get
from json import dumps
import json
from urllib.parse import urlencode


In [2]:
pwd = os.getcwd()
filepath = (pwd + f"/data_sets/")

In [16]:
ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"
AREA_TYPE = 'nation'
AREA_NAME = 'england'

In [19]:
filters = [f"areaType={AREA_TYPE}", f"areaName={AREA_NAME}"]

# Total Case Metrics Up To Todays Date
total_case_data_structure = {
    "date":"date",
    "area":"areaName",
    "total_cases":"cumCasesByPublishDate",
    "case_rate_per_100k":"cumCasesByPublishDateRate",
    "death_28_days":"cumDeaths28DaysByPublishDate",
    "death_rate":"cumDeaths28DaysByPublishDateRate",
    "cum_male_cases":"maleCases",
    "cum_female_cases":"femaleCases"
}

# Total Current Hospital Data including total admissions and current admisions
hospital_data_structure = {

    "date":"date",
    "area":"areaName",
    "patients_in_hospital":"hospitalCases",
    "people_on_vents":"covidOccupiedMVBeds",
    "total_hospitalisations":'cumAdmissions',
    "hos_by_age":"cumAdmissionsByAge", 

}

# Due to this data only producing 1 Dates worth of breakdown - Ive put it on its own structure - makes it easier to break down later
demographic_sex_death_data_structure = {

    "date":"date", 
    "area":"areaName",
    "female_28_days":"femaleDeaths",
    "male_deaths_28_days":"maleDeaths"
}

# New Cases From day of request
new_case_data_structure = {
    "date":"data",
    "area":"areaName",
    "new_cases":"newCasesByPublishDate",
    "new_change":"newCasesByPublishDateChange",
    "new_percentage_change":"newCasesByPublishDatePercentageChange",
    "new_cases_rolling_rate":"newCasesByPublishDateRollingRate",
    "new_deaths":"newDeath28DaysByPublishDate",
    "new_d_change":"newDeath28DaysByPublishDateChange",
    "new_d_percentage_change":"newDeaths28DaysByPublishDateChangePercentage" ,
    "new_d_rolling_rate":"newDeaths28DaysByDeathRollingRate",
    "new_d_by_age":"newDeaths28DaysByDeathDateAgeDemographics"
}
# Set API_params via calls to functions. 
api_params = {
    "filters": str.join(";", filters),
    
}
formats = {
    "json"
}

data_structures = {"total_case_data" : total_case_data_structure, "hospital_data" : hospital_data_structure, "demographic_sex_death_data" : demographic_sex_death_data_structure, "new_case_data" : new_case_data_structure}

encoded_params = urlencode(api_params)



In [35]:
def get_data():
   for key, value in data_structures.items():
      api_params["structure"] = dumps(value, separators=(",", ":"))
      #print(api_params["structure"])
      (f"Data being processed for: {key}")
      for fmt in formats:
         api_params["format"] = fmt
         response = get(ENDPOINT, params=api_params, timeout=10)
         #print(response.url)
         if response.status_code >= 400:
            raise RuntimeError(f'Request Failed: {response.text}')

         #print(response.url)
         try:
            data = response.json()
         except ValueError:
            pass
      return data

In [40]:
def save_to_file(name='', fmat=''):
    with open(filepath + name + fmat, 'a') as file:
        json.dump(get_data(), file, indent=4)

In [41]:
for key in data_structures:

    save_to_file(key, '.json')